# 📊 Exploratory Data Analysis (EDA)
## Hull Tactical Market Prediction - Phase 1

**Date:** 2025-11-11

**Objectives:**
1. Feature Group Analysis (M/E/I/P/V/S/D)
2. Target Variable Time Series Characteristics  
3. Correlation Analysis
4. Market Regime Identification
5. Train/Test Distribution Shift Detection
6. Feature Stationarity Assessment
7. Benchmark Calculation

## 1. Setup and Imports

In [ ]:
# Standard libraries
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Get project root (go up one level from notebooks folder)
notebook_dir = os.getcwd() if '__file__' not in dir() else os.path.dirname(os.path.abspath(__file__))
project_root = os.path.dirname(notebook_dir) if 'notebooks' in notebook_dir else notebook_dir
sys.path.insert(0, project_root)

# Data manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical tests
from scipy import stats
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Clustering
from sklearn.cluster import KMeans

# Project utilities
from src.data import DataLoader, calculate_benchmark_score
from src.utils import set_seed, load_config

# Load configuration (using absolute path from project root)
config_path = os.path.join(project_root, 'conf', 'params.yaml')
config = load_config(config_path)
print(f"📋 Loaded config from: {config_path}")
print(f"📋 Seed: {config['seed']}")

# Set random seed from config
set_seed(config['seed'])

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

# Plot settings  
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

print('✅ Setup complete!')

## 2. Load Data

In [ ]:
# Change working directory to project root
os.chdir(project_root)
print(f"📂 Working directory: {os.getcwd()}")

# Initialize DataLoader (now it can find conf/params.yaml)
loader = DataLoader()

# Load data
train_df, test_df = loader.load_data()

print(f'📦 Train: {train_df.shape}, Test: {test_df.shape}')
print(f'📅 Date range: {train_df["date_id"].min()} - {train_df["date_id"].max()}')

# Display feature groups
print('\n🏷️ Feature Groups:')
for group, features in loader.feature_groups.items():
    print(f'  {group}: {len(features)} features')

train_df.head()

## 3. Data Quality Check

In [ ]:
# Missing value comprehensive analysis
print("="*60)
print("MISSING VALUE ANALYSIS")
print("="*60)
loader = DataLoader()
train_df, _ = loader.load_data()
# 1. Get missing summary from DataLoader
missing_summary = loader.get_missing_summary(train_df)
print("\n1. Missing Summary (first 20 features):")
print(missing_summary.head(20))

# 2. Check for intermittent missing values
print("\n2. Intermittent Missing Detection:")
print("-" * 60)

features_with_intermittent = []
features_initial_only = []

for col in train_df.columns:
    if col in ['date_id', 'is_scored', 'forward_returns']:
        continue
    
    series = train_df[col]
    
    # Find first valid index : 시리즈에서 첫 번째로 '유효한 값(= NaN이 아닌 값)'이 등장하는 index(행 번호)를 찾는 함수
    first_valid = series.first_valid_index()
    
    if first_valid is None:
        continue  # Completely missing column
    
    # Check if there are any NaN values AFTER first valid index
    after_first_valid = series.loc[first_valid:]
    has_intermittent = after_first_valid.isna().any() # 결측치가 존재하면 True, 없으면 False
    
    if has_intermittent:
        features_with_intermittent.append(col)
    else:
        features_initial_only.append(col)

print(f"Features with intermittent missing: {len(features_with_intermittent)}")
print(f"Features with initial-only missing: {len(features_initial_only)}")

if features_with_intermittent:
    print(f"\n⚠️ Features with intermittent gaps (first 10):")
    for feat in features_with_intermittent[:10]:
        print(f"  - {feat}")
else:
    print("\n✅ No intermittent missing values found - all missing is initial only!")

# 3. Visualize missing patterns for top features
print("\n3. Missing Pattern Visualization (top 5 features by missing %):")
print("-" * 60)

top_missing = missing_summary.nlargest(5, 'missing_pct') # nlargest : missing_pct 칼럼에서 값이 가장 큰 n개의 행을 내림차순으로 반환.

fig, axes = plt.subplots(len(top_missing), 1, figsize=(16, 3*len(top_missing)))
if len(top_missing) == 1:
    axes = [axes]

for idx, (_, row) in enumerate(top_missing.iterrows()):
    feat = row['feature']
    series = train_df[feat]
    
    # Create binary mask (1=valid, 0=missing)
    # ~series.isna() : 값 존재 여부, astype(int) : bool 값을 int형으로 변환
    mask = (~series.isna()).astype(int)
    
    axes[idx].plot(train_df['date_id'], mask, linewidth=0.5)
    axes[idx].set_title(f"{feat} - {row['missing_pct']:.1f}% missing")
    axes[idx].set_ylabel('Valid (1) / Missing (0)')
    axes[idx].set_ylim(-0.1, 1.1)
    axes[idx].grid(True, alpha=0.3)
    
    # Mark first and last valid indices
    if row['first_valid_idx'] >= 0:
        first_date = train_df.loc[row['first_valid_idx'], 'date_id']
        axes[idx].axvline(x=first_date, color='green', linestyle='--', 
                         alpha=0.5, label=f'First valid: {first_date}')
    if row['last_valid_idx'] >= 0:
        last_date = train_df.loc[row['last_valid_idx'], 'date_id']
        axes[idx].axvline(x=last_date, color='red', linestyle='--', 
                         alpha=0.5, label=f'Last valid: {last_date}')
    axes[idx].legend(loc='upper left')

axes[-1].set_xlabel('Date ID')
plt.tight_layout()
plt.show()

# 결측치 많은 칼럼 5개 시각화.
print("\n✅ Missing value analysis complete!")

## 4. Feature Group Analysis

In [ ]:
# Analyze each feature group
for group, features in loader.feature_groups.items():
    if not features or group == 'D':
        continue
    
    print(f"\n{'='*60}\n{group} Features ({len(features)})\n{'='*60}")
    
    # Descriptive statistics : count, mean, std, min, 25%, 50%, 75%, max
    desc = train_df[features].describe()
    print(desc)
    
    # Plot sample distributions
    sample_features = features[:min(4, len(features))]
    fig, axes = plt.subplots(1, len(sample_features), figsize=(16, 4))
    if len(sample_features) == 1:
        axes = [axes]
    
    for idx, feat in enumerate(sample_features):
        train_df[feat].hist(bins=50, ax=axes[idx], alpha=0.7) # 하나의 칼럼 데이터의 분포를 50개 구간으로 나눈 그래프
        axes[idx].set_title(f'{feat} Distribution')
    
    plt.suptitle(f'{group} Group - Sample Distributions')
    plt.tight_layout()
    plt.show()

## 5. Target Variable Analysis

In [ ]:
target = 'forward_returns'  # S&P500의 다음날 수익률

# Time series plot
plt.figure(figsize=(16, 6))
plt.plot(train_df['date_id'], train_df[target], alpha=0.6, linewidth=0.5)
plt.title('Forward Returns Over Time')
plt.xlabel('Date ID')
plt.ylabel('Forward Returns')
plt.grid(True, alpha=0.3)
plt.show()

# Distribution plots
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Histogram
axes[0].hist(train_df[target].dropna(), bins=100, alpha=0.7, edgecolor='black')
axes[0].set_title('Distribution')
axes[0].set_xlabel('Forward Returns')

# Q-Q plot
stats.probplot(train_df[target].dropna(), dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot')

# Box plot
axes[2].boxplot(train_df[target].dropna())
axes[2].set_title('Box Plot')
axes[2].set_ylabel('Forward Returns')

plt.tight_layout()
plt.show()

# Statistics
print(f"Mean: {train_df[target].mean():.6f}")
print(f"Std: {train_df[target].std():.6f}")
print(f"Skewness: {stats.skew(train_df[target].dropna()):.4f}")
print(f"Kurtosis: {stats.kurtosis(train_df[target].dropna()):.4f}")

## 6. Autocorrelation Analysis

In [ ]:
# 시계열 데이터(회귀)의 성질을 분석하는데 활용되는 상관도표(Correlogram) : ACF and PACF plots
# x가 0일때는 y는 무조건 1, 그 외 x값인경우, y는 0이랑 가까울수록 좋다. 
# -> 0이랑 가까울 수록 독립적. (파란영역 안에 들어와야 좋음.)
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# ACF (Autocorrelation Function, 자기 상관 함수) : 전체 상관 측정
plot_acf(train_df[target].dropna(), lags=50, ax=axes[0])
axes[0].set_title('Autocorrelation Function (ACF)')

# PSCF (Partial Autocorrelation Function, 편자기상관함수) : 개별 시차별 자기 상관 측정
plot_pacf(train_df[target].dropna(), lags=50, ax=axes[1])
axes[1].set_title('Partial Autocorrelation Function (PACF)')

plt.tight_layout()
plt.show()

# Print significant lags
acf_vals = acf(train_df[target].dropna(), nlags=20)
print("Significant ACF lags (|r| > 0.05):")
for i, val in enumerate(acf_vals[1:], 1):
    if abs(val) > 0.05:
        print(f"  Lag {i}: {val:.4f}")

## 7. Correlation Analysis

In [ ]:
# Sample features for correlation
sample_features = []
for group, features in loader.feature_groups.items():
    if group != 'D' and features:
        sample_features.extend(features[:min(5, len(features))])

sample_features.append(target)

# Correlation matrix
corr_matrix = train_df[sample_features].corr()

# Plot heatmap
plt.figure(figsize=(14, 12))
sns.heatmap(corr_matrix, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, vmin=-1, vmax=1)
plt.title('Feature Correlation Heatmap (Sample)')
plt.tight_layout()
plt.show()

# Top correlations with target
target_corr = corr_matrix[target].drop(target).sort_values(ascending=False)
print("\nTop 10 Positive Correlations with Target:")
print(target_corr.head(10))
print("\nTop 10 Negative Correlations with Target:")
print(target_corr.tail(10))

## 8. Market Regime Analysis

현재 노트북에서는 forward_returns 열과 다른 피처 그룹(예: V로 시작하는 열들)에 대해 롤링 변동성을 계산하고 있습니다. 이때 윈도우 크기는 설정 파일(config['model_risk']['risk_window'])에서 정의된 값을 사용합니다. 롤링 변동성은 데이터의 고변동(high volatility) 또는 저변동(low volatility) 구간을 식별하는 데 유용하며, 시장 상태 분석이나 리스크 평가에 자주 활용됩니다.


In [ ]:
# Calculate rolling volatility (using config window)
window = config['model_risk']['risk_window']
train_df['rolling_vol'] = train_df[target].rolling(window).std()

# Identify regimes (quartiles)
vol_quartiles = train_df['rolling_vol'].quantile([0.25, 0.5, 0.75])
print(f"Volatility Quartiles (window={window}):\n{vol_quartiles}")

# Classify regimes
def classify_regime(vol):
    if pd.isna(vol):
        return 'Unknown'
    elif vol < vol_quartiles[0.25]:
        return 'Low Vol'
    elif vol < vol_quartiles[0.75]:
        return 'Medium Vol'
    else:
        return 'High Vol'

train_df['regime'] = train_df['rolling_vol'].apply(classify_regime)

# Plot regimes
fig, axes = plt.subplots(2, 1, figsize=(16, 10), sharex=True)

# Returns with regime coloring
for regime in ['Low Vol', 'Medium Vol', 'High Vol']:
    mask = train_df['regime'] == regime
    axes[0].scatter(train_df.loc[mask, 'date_id'], 
                   train_df.loc[mask, target],
                   label=regime, alpha=0.5, s=1)
axes[0].set_ylabel('Forward Returns')
axes[0].legend()
axes[0].set_title('Returns by Volatility Regime')
axes[0].grid(True, alpha=0.3)

# Rolling volatility
axes[1].plot(train_df['date_id'], train_df['rolling_vol'], linewidth=1)
axes[1].axhline(vol_quartiles[0.25], color='g', linestyle='--', label='Q1')
axes[1].axhline(vol_quartiles[0.75], color='r', linestyle='--', label='Q3')
axes[1].set_xlabel('Date ID')
axes[1].set_ylabel('Rolling Volatility')
axes[1].legend()
axes[1].set_title(f'Rolling Volatility ({window}-day window)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Performance by regime
print("\nPerformance by Regime:")
regime_stats = train_df.groupby('regime')[target].agg(['count', 'mean', 'std', 'min', 'max'])
print(regime_stats)

In [ ]:
# Compare rolling volatility of V columns with forward_returns
v_columns = [col for col in train_df.columns if col.startswith('V')]
rolling_vol_v = train_df[v_columns].rolling(window).std()

# Plot comparison
fig, axes = plt.subplots(len(v_columns) + 1, 1, figsize=(16, 5 * (len(v_columns) + 1)), sharex=True)

# Rolling volatility of forward_returns
axes[0].plot(train_df['date_id'], train_df['rolling_vol'], linewidth=1, label='Forward Returns Volatility')
axes[0].set_title('Rolling Volatility Comparison')
axes[0].set_ylabel('Volatility')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Rolling volatility of V columns
for idx, col in enumerate(v_columns):
    axes[idx + 1].plot(train_df['date_id'], rolling_vol_v[col], linewidth=1, label=f'{col} Volatility')
    axes[idx + 1].set_ylabel('Volatility')
    axes[idx + 1].legend()
    axes[idx + 1].grid(True, alpha=0.3)

axes[-1].set_xlabel('Date ID')
plt.tight_layout()
plt.show()

## 9. Stationarity Tests

이 셀은 Augmented Dickey-Fuller (ADF) 테스트를 사용하여 특정 피처(feature)들이 
**정상성(stationarity)**을 가지는지 평가합니다. 정상성은 시계열 데이터 분석에서 중요한 개념으로, 
데이터의 평균과 분산이 시간에 따라 일정한지를 나타냅니다. 아래는 코드의 주요 부분과 그 의미를 단계별로 설명합니다:

In [ ]:
# Test stationarity for target and sample features
test_features = [target] + [f for g, feats in loader.feature_groups.items() 
                            for f in feats[:2] if g != 'D' and feats][:10] # 전체 다 하려면 [:2], [:10] 제거

stationarity_results = []

for feat in test_features:
    series = train_df[feat].dropna()
    
    if len(series) < 50:
        continue
    
    # ADF test
    '''
    * 각 피처에 대해 다음을 수행합니다:
    1. 결측값 제거: dropna()를 사용하여 결측값을 제거합니다.
    2. 샘플 크기 확인: 데이터가 50개 미만인 경우 테스트를 건너뜁니다.
    3. ADF 테스트 실행: adfuller 함수를 사용하여 ADF 테스트를 수행합니다.
    4. adfuller는 정상성을 평가하는 통계량과 p-value를 반환합니다.
    5. 결과 저장: 피처 이름, ADF 통계량, p-value, 정상성 여부를 저장합니다.
    '''
    adf_result = adfuller(series, autolag='AIC')
    
    stationarity_results.append({
        'Feature': feat,
        'ADF Statistic': adf_result[0],
        'ADF p-value': adf_result[1],
        'Stationary (p<0.05)': adf_result[1] < 0.05
    })

stationarity_df = pd.DataFrame(stationarity_results)
print("Stationarity Test Results (ADF):")
print(stationarity_df.sort_values('ADF p-value'))

# Count stationary vs non-stationary
n_stationary = stationarity_df['Stationary (p<0.05)'].sum()
n_total = len(stationarity_df)
print(f"\nStationary: {n_stationary}/{n_total} ({n_stationary/n_total*100:.1f}%)") 

## 10. Train/Test Distribution Shift

이 셀은 Kolmogorov-Smirnov (KS) 테스트를 사용하여 훈련 데이터(train_df)와 테스트 데이터(test_df) 간의 특징 분포 차이를 분석합니다. KS 테스트는 두 데이터 분포가 동일한지 여부를 평가하는 비모수적 통계 테스트입니다. 아래는 코드의 주요 부분과 그 의미를 단계별로 설명합니다.

In [ ]:
# Compare distributions for common features
common_features = [c for c in train_df.columns if c in test_df.columns 
                   and c not in ['date_id', 'is_scored']]

# Sample features to test
test_sample = [f for f in common_features if not f.startswith('lagged')][:15]

shift_results = []

for feat in test_sample:
    train_vals = train_df[feat].dropna()
    test_vals = test_df[feat].dropna()
    
    if len(train_vals) < 10 or len(test_vals) < 5:
        continue
    
    # KS test
    ks_stat, ks_pval = ks_2samp(train_vals, test_vals)
    
    shift_results.append({
        'Feature': feat,
        'KS Statistic': ks_stat,
        'KS p-value': ks_pval,
        'Significant Shift (p<0.05)': ks_pval < 0.05
    })

shift_df = pd.DataFrame(shift_results)
print("Distribution Shift Analysis (KS Test):")
print(shift_df.sort_values('KS Statistic', ascending=False))

# Count shifts
n_shifts = shift_df['Significant Shift (p<0.05)'].sum()
print(f"\nFeatures with significant shift: {n_shifts}/{len(shift_df)}")

## 11. Benchmark Calculation

In [ ]:
# Calculate benchmark with allocation=1.0
score, returns, metrics = calculate_benchmark_score(train_df, allocation=1.0)

print("=" * 60)
print("BENCHMARK METRICS (Allocation = 1.0)")
print("=" * 60)
print(f"\nScore: {score:.4f}")
print(f"Sharpe Ratio: {metrics['sharpe']:.4f}")
print(f"Annual Sharpe: {metrics['annual_sharpe']:.4f}")
print(f"Volatility Penalty: {metrics['vol_penalty']:.4f}")
print(f"\nAnnual Return: {metrics['annual_return']:.2%}")
print(f"Annual Volatility: {metrics['annual_vol']:.2%}")
print(f"Max Drawdown: {metrics['max_drawdown']:.2%}")
print(f"Win Rate: {metrics['win_rate']:.2%}")

# Cumulative returns plot
cumulative_returns = (1 + returns).cumprod()

fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Cumulative returns
axes[0].plot(cumulative_returns.values, linewidth=1.5)
axes[0].set_title('Benchmark Strategy - Cumulative Returns (Allocation=1.0)')
axes[0].set_ylabel('Cumulative Return')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=1.0, color='r', linestyle='--', alpha=0.5)

# Drawdown
running_max = cumulative_returns.expanding().max()
drawdown = (cumulative_returns - running_max) / running_max

axes[1].fill_between(range(len(drawdown)), drawdown.values, 0, alpha=0.3, color='red')
axes[1].plot(drawdown.values, linewidth=1, color='darkred')
axes[1].set_title('Drawdown')
axes[1].set_xlabel('Days')
axes[1].set_ylabel('Drawdown')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 12. Summary and Insights

In [ ]:
print("="*60)
print("EDA SUMMARY")
print("="*60)

print(f"\n📦 Data Shape:")
print(f"  Train: {train_df.shape}")
print(f"  Test: {test_df.shape}")

print(f"\n🏷️ Feature Groups:")
for group, features in loader.feature_groups.items():
    print(f"  {group}: {len(features)} features")

print(f"\n📊 Target Statistics:")
print(f"  Mean: {train_df[target].mean():.6f}")
print(f"  Std: {train_df[target].std():.6f}")
print(f"  Sharpe: {train_df[target].mean()/train_df[target].std():.4f}")

print(f"\n🎯 Benchmark Performance:")
print(f"  Score: {score:.4f}")
print(f"  Annual Sharpe: {metrics['annual_sharpe']:.4f}")
print(f"  Max Drawdown: {metrics['max_drawdown']:.2%}")

print(f"\n⚠️ Key Findings:")
print(f"  - Stationary features: {n_stationary}/{n_total}")
print(f"  - Distribution shifts: {n_shifts}/{len(shift_df)}")
print(f"  - Missing data: See quality check above")

print("\n✅ EDA Complete!")